In [35]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import shutil
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [36]:
label_mapping = {
    'Cell': 0,
    'Cell-Multi': 1,
    'Cracking': 2,
    'Hot-Spot': 3,
    'Hot-Spot-Multi': 4,
    'Shadowing': 5,
    'Diode': 6,
    'Diode-Multi': 7,
    'Vegetation': 8,
    'Soiling': 9,
    'No-Anomaly' : 10,
    'other' : 11
}

In [37]:
inv_label_mapping = {v: k for k, v in label_mapping.items()}

In [38]:
class TestDataset(Dataset):
    def __init__(self, labels, transform=None):
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.labels[idx]['image_filepath']
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

In [39]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [55]:
with open(r'dataset\test_labels.json', 'r') as f:
    test_data = json.load(f)

In [41]:
your_test_labels_list = [{'image_filepath': v['image_filepath']} for k, v in test_data.items()]

In [42]:
model = models.densenet121(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, len(label_mapping))  

c:\Users\win11\anaconda3\envs\cv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\win11\anaconda3\envs\cv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [43]:
model.load_state_dict(torch.load('fine_tuned_densenet121.pth'))

C:\Users\win11\AppData\Local\Temp\ipykernel_31612\3857100851.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fine_tuned_densenet121.pth

<All keys matched successfully>

In [44]:
criterion = nn.CrossEntropyLoss()

In [45]:
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [47]:
test_dataset = TestDataset(labels=your_test_labels_list, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [48]:
all_labels = []
all_preds = []

In [49]:
test_loss = 0.0
correct = 0
total = 0

In [56]:
with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        
        dummy_labels = [0] * len(predicted) 
        all_labels.extend(dummy_labels)

        loss = criterion(outputs, torch.tensor(dummy_labels).to(device))
        test_loss += loss.item()

        total += len(predicted)
        correct += (predicted == torch.tensor(dummy_labels).to(device)).sum().item()

In [57]:
avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total
print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 11.1255, Test Accuracy: 0.0728
